# fpd_data_processing tutorial

This notebook shows how to use the `fpd_data_processing` library to analyse pixelated scanning transmission electron microscopy (STEM) data, and differential phase contrast (DPC) data.

More documentation is found at https://fast_pixelated_detectors.gitlab.io/fpd_data_processing/

## Importing libraries

The first step is setting the plotting toolkit

In [ ]:
%matplotlib nbagg

Then import the library itself

You might get a "WARNING:hyperspy_gui_traitsui", this can be ignored.

In [ ]:
import fpd_data_processing.api as fp

## Working with fast pixelated detector STEM data

### Loading data

In [ ]:
s = fp.load_fpd_signal

For large files, use `lazy=True`: `s = fp.load_fpd_signal("data.hspy", lazy=True)`

Here, we use an artificial dataset, which is found in `fp.dummy_data`

In [ ]:
s = fp.dummy_data.get_holz_heterostructure_test_signal()

This returns a `PixelatedSTEM` class, which is inherits HyperSpy's `Signal2D`. So all functions which work in `Signal2D`, also works here:

In [ ]:
s

In [ ]:
s.plot()

### Radial integration

A common task is getting the intensity as a function of scattering angle. This is done using radial integration, which firstly requires finding the center of the electron beam. Here we use the `center_of_mass` function.

In [ ]:
s_com = s.center_of_mass(threshold=1.)

This returns a `DPCSignal2D` class, which will be explored more later. What we need to know is that is it basically a HyperSpy `Signal2D` class, where the x-beam shifts are in the first navigation index (`s.inav[0]`), while the y-shifts are in the second navigation index (`s.inav[1]`).

In [ ]:
s_com.plot()

To do the radial integration itself, use the `radial_integration` method, which requires the `centre_x` and `centre_y` arguments to be specified.

In [ ]:
s_radial = s.radial_integration(centre_x=s_com.inav[0].data, centre_y=s_com.inav[1].data)

This returns a new signal, where the signal dimensions has been reduced from 2 to 1 dimensions. This is especially useful when working with large datasets, where this operation can drastically reduce the data size, making it possible to load the full data into memory.

In [ ]:
s_radial

Plotting it shows the electron scattering for each probe position:

In [ ]:
s_radial.plot()

To rather visualize the data as function of scattering angle (essentially virtual annular dark field), we can transpose the data using `s_radial.T`. This "flips" the signal and navigation axes:

In [ ]:
s_radial.T.plot()

## Differential phase contrast (DPC) signals

These signal classes are used for beam shift datasets, where x-shifts are stored in the first navigation index (`s_dpc.inav[0]`) and the y-shifts in the second navigation index (`s_dpc.inav[1]`).

They contain many different methods for both processing and visualizing DPC data.

Here, we again use `fp.dummy_data` to get a signal to work with.

There types of signals can be loaded using `s = fp.load_dpc_signal`.

In [ ]:
s_dpc = fp.dummy_data.get_square_dpc_signal(add_ramp=True)

In [ ]:
s_dpc.plot()

### Correcting d-scan (ramp)

The `s_dpc` has a lot of d-scan, to correct it use the `correct_ramp` method. This function is fairly basic, with only the possibility to subtract a linear ramp.

In [ ]:
s_dpc = s_dpc.correct_ramp(corner_size=0.05)

In [ ]:
s_dpc.plot()

### Plotting methods

The class also has several methods for visualizing DPC data: `get_color_signal`, `get_magnitude_signal` and `get_color_image_with_indicator`.

The two former returns a HyperSpy signal, while the latter interfaces directly with the matplotlib backend making it more customizable.

In [ ]:
s_dpc.get_color_signal?

In [ ]:
s_color = s_dpc.get_color_signal()
s_color.plot()

The `get_color_signal` method has a `rotation` argument, which is used to correct for mismatch between the scan direction and diffraction rotation.

In [ ]:
s_color_rot = s_dpc.get_color_signal(rotation=45)
s_color_rot.plot()

`get_magnitude_signal` gives the magnitude of the beam shift vector

In [ ]:
s_magnitude = s_dpc.get_magnitude_signal()
s_magnitude.plot()

The `get_color_image_with_indicator` method has a large degree of customizability, which is useful when making images for presentations, posters or articles.

By default it returns a matplotlib figure object, which can be saved directly

In [ ]:
fig = s_dpc.get_color_image_with_indicator()

In [ ]:
fig.savefig("dpc_image.jpg")

In [ ]:
s_dpc.get_color_image_with_indicator?

It also accepts a matplotlib subplot object, which makes it easy to integrate into larger figures.

An example using more of the customizability

In [ ]:
import matplotlib.pyplot as plt
fig, axarr = plt.subplots(1, 2, figsize=(8, 4))
ax_dpc = axarr[0]
ax_dif = axarr[1]
ax_dif.imshow(s.inav[0, 0].data)
s_dpc.get_color_image_with_indicator(indicator_rotation=90, scalebar_size=30, ax=ax_dpc)
fig.savefig("dpc_figure.jpg")

## Various rotating

Rotating the scan dimensions

In [ ]:
s_dpc_rot = s_dpc.rotate_data(20)
s_dpc_rot.get_color_signal().plot()

Rotating the beam shifts, to correct for mismatch between the scan direction and diffraction rotation.

In [ ]:
s_dpc_rot = s_dpc.rotate_beam_shifts(25)
s_dpc_rot.get_color_signal().plot()

## Blurring the beam shifts

In [ ]:
s_dpc_blur = s_dpc.gaussian_blur(1.2)
s_dpc_blur.get_color_signal().plot()

## Bivariate histogram

In [ ]:
s_hist = s_dpc.get_bivariate_histogram()
s_hist.plot(cmap='viridis')